In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
dataset = "C:/Users/User/Downloads/ukr-eng/ukr.txt"
batch_size = 64
epochs = 40
latent_dim = 256
num_samples = 10000

In [3]:
text_en = []
text_ukr = []
chars_en = set()
chars_ukr = set()
with open(dataset, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    text_en.append(input_text)
    text_ukr.append(target_text)
    for ch in input_text:
        if ch not in chars_en:
            chars_en.add(ch)
    for ch in target_text:
        if ch not in chars_ukr:
            chars_ukr.add(ch)

In [4]:
for _ in range(5):
    print(text_en[_])
    print(text_ukr[_])

Go.
	Йди.

Hi.
	Вітаю!

Hi.
	Привіт.

Hi.
	Привіт!

Run!
	Біжіть!



In [5]:
chars_en = sorted(list(chars_en))
chars_ukr = sorted(list(chars_ukr))
num_encoder_tokens = len(chars_en)
num_decoder_tokens = len(chars_ukr)
max_encoder_seq_length = max([len(txt) for txt in text_en])
max_decoder_seq_length = max([len(txt) for txt in text_ukr])

#input_token_index = dict([(ch, i) for i, ch in enumerate(chars_en)])
#target_token_index = dict([(ch, i) for i, ch in enumerate(chars_ukr)])
input_token_index = {ch: i for i, ch in enumerate(chars_en)}
target_token_index = {ch: i for i, ch in enumerate(chars_ukr)}

encoder_input_data = np.zeros((len(text_en), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(text_en), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
 
for i, (en, ukr) in enumerate(zip(text_en, text_ukr)):
    # Process English text
    for t, ch in enumerate(en):
        encoder_input_data[i, t, input_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    
    # Process Ukrainian text
    for t, ch in enumerate(ukr):
        decoder_input_data[i, t, target_token_index[ch]] = 1.0
        # Set target data for all characters except the first one
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[ch]] = 1.0
    # Fill remaining spaces with padding character
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [6]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [7]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save("C:/Users/User/Downloads/s2s.keras")

Epoch 1/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 200ms/step - accuracy: 0.7404 - loss: 1.2770 - val_accuracy: 0.7310 - val_loss: 0.9893
Epoch 2/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 195ms/step - accuracy: 0.7714 - loss: 0.8553 - val_accuracy: 0.7690 - val_loss: 0.8570
Epoch 3/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 188ms/step - accuracy: 0.7965 - loss: 0.7984 - val_accuracy: 0.7713 - val_loss: 0.8778
Epoch 4/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 196ms/step - accuracy: 0.8083 - loss: 0.7114 - val_accuracy: 0.8027 - val_loss: 0.7158
Epoch 5/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 204ms/step - accuracy: 0.8182 - loss: 0.6448 - val_accuracy: 0.8088 - val_loss: 0.6786
Epoch 6/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 194ms/step - accuracy: 0.8245 - loss: 0.6147 - val_accuracy: 0.8126 - val_loss: 0.6528
Epoch 7/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 25s 200ms/step - accuracy: 0.8292 - loss: 0.5929 - val_accuracy: 0.8159 - val_loss: 0.6391
Epoch 8/40
125/125 ━━━━━━━━━━━━━━━━━━━━ 25s 201ms/step - accuracy: 0.8333 - loss: 0

In [8]:
model = keras.models.load_model("C:/Users/User/Downloads/s2s.keras")

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output 
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [9]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [10]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", text_en[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
-
Input sentence: Go.
Decoded sentence: Заможуйте!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
-
Input sentence: Hi.
Decoded sentence: Це було!

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━